In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
file = open("Beatles.txt","r")
lyrics = file.readlines()
file.close()

In [3]:
def create_vocab(lyrics):
    word_to_i = {}
    i_to_word = {}
    ind = 0
    for i in lyrics:
        for j in i.split():
            if j in word_to_i:
                continue
            else:
                word_to_i[j] = ind
                i_to_word[ind] = j
                ind+=1
    word_to_i['<EOS>'] = ind
    i_to_word[ind] = '<EOS>'
    ind+=1
    word_to_i['<PAD>'] = ind
    i_to_word[ind] = '<PAD>'
    return word_to_i,i_to_word

In [4]:
word_to_i,i_to_word = create_vocab(lyrics)

In [5]:
def create_vectors(lyrics,word_to_i):
    X = []
    Y = []
    max_val = 15
    for i in lyrics:
        temp_x = []
        temp_y = []
        words = i.split()
        if len(words)>max_val-1:
            words = words[0:max_val-1]
        for j in range(len(words)-1):
            temp_x.append(word_to_i[words[j]])
            temp_y.append(word_to_i[words[j+1]])
        temp_x.append(word_to_i[words[len(words)-1]])
        temp_x.append(word_to_i['<EOS>'])
        temp_y.append(word_to_i['<EOS>'])
        temp_y.append(word_to_i['<PAD>'])
        pad = [word_to_i['<PAD>'] for _ in range(max_val-len(words)-1)]
        temp_x+=pad
        temp_y+=pad
        X.append(temp_x)
        Y.append(temp_y)
    return X,Y
            

In [6]:
X,Y = create_vectors(lyrics,word_to_i)
X = np.array(X)
Y = np.array(Y)

In [7]:
class BeatlesDataset(Dataset):
    def __init__(self,X,Y):
        self.X = X
        self.Y = Y
    def __getitem__(self,idx):
        return (torch.tensor(X[idx]).to(torch.int64),torch.tensor(Y[idx]).to(torch.int64))
    def __len__(self):
        return len(self.X)
    

In [8]:
data = BeatlesDataset(X,Y)

In [9]:
dataloader = DataLoader(data,batch_size=8,shuffle=True,drop_last=True)

In [10]:
class TextGenerator(nn.Module):
    def __init__(self):
        super(TextGenerator,self).__init__()
        self.lstm_size = 256
        self.embedding_size = 300
        self.num_layers = 3
        vocab_len = 2240
        
        self.embedding = nn.Embedding(num_embeddings=vocab_len,embedding_dim=self.embedding_size)
        self.lstm = nn.LSTM(input_size=self.embedding_size,hidden_size=self.lstm_size,num_layers=self.num_layers,dropout=0.2,batch_first=True)
        self.fc = nn.Linear(self.lstm_size,vocab_len)
        
    def forward(self,X,prev_state):
        embed = self.embedding(X)
        output, state = self.lstm(embed,prev_state)
        logits = self.fc(output)
        return logits,state
    
    def init_state(self, batch_size):
        return (torch.zeros(self.num_layers,batch_size,self.lstm_size),
                torch.zeros(self.num_layers,batch_size,self.lstm_size))

In [11]:
def trainGenerator(dataloader,generator):
    generator.train()
    epochs = 30
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(generator.parameters(), lr=0.0005)
    decayRate = 0.96
    my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)
    for epoch in range(epochs):
        state_h, state_c = generator.init_state(8)
        for batch, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()
            y_pred, (state_h, state_c) = generator(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)
            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()

        print({ 'epoch': epoch, 'loss': loss.item() })

In [12]:
generator = TextGenerator()
trainGenerator(dataloader,generator)

{'epoch': 0, 'loss': 3.5135185718536377}
{'epoch': 1, 'loss': 2.240377902984619}
{'epoch': 2, 'loss': 3.119553804397583}
{'epoch': 3, 'loss': 2.7091763019561768}
{'epoch': 4, 'loss': 3.153564929962158}
{'epoch': 5, 'loss': 2.7334821224212646}
{'epoch': 6, 'loss': 1.9841578006744385}
{'epoch': 7, 'loss': 2.224437713623047}
{'epoch': 8, 'loss': 1.826805591583252}
{'epoch': 9, 'loss': 1.7699040174484253}
{'epoch': 10, 'loss': 1.7331290245056152}
{'epoch': 11, 'loss': 1.8287843465805054}
{'epoch': 12, 'loss': 2.4480490684509277}
{'epoch': 13, 'loss': 2.348189115524292}
{'epoch': 14, 'loss': 1.7935538291931152}
{'epoch': 15, 'loss': 1.3278963565826416}
{'epoch': 16, 'loss': 2.018045425415039}
{'epoch': 17, 'loss': 1.643039584159851}
{'epoch': 18, 'loss': 1.7255892753601074}
{'epoch': 19, 'loss': 0.9781566858291626}
{'epoch': 20, 'loss': 1.4304991960525513}
{'epoch': 21, 'loss': 1.2370593547821045}
{'epoch': 22, 'loss': 1.198279619216919}
{'epoch': 23, 'loss': 1.2211267948150635}
{'epoch': 2

In [22]:
def predict(i_to_word,word_to_i,generator,text):
    generator.eval()
    words = text.split()
    state_h,state_c = generator.init_state(1)
    for i in range(0,10):
        x = torch.tensor([[word_to_i[w] for w in words]])
        y_pred, (state_h, state_c) = generator(x, (state_h, state_c))
                
        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(i_to_word[word_index])
    return words

In [23]:
def songmaker(i_to_word,word_to_i,generator,text):
    initial = text
    for i in range(15):
        words = predict(i_to_word,word_to_i,generator,initial)
        if '<EOS>' in words:
            words = words[:words.index('<EOS>')]
        print(' '.join(words))
        initial = ' '.join(words[-3:])